In [21]:
import os, glob
import csv
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
import nibabel as nib
from scipy import stats
from nibabel.testing import data_path

# csv write a formated csv file
def csvFormatedOutput(fileName, ans):
    with open(fileName, 'w', newline='') as f:
        c = csv.writer(f, delimiter=',',
                       quotechar='|', quoting=csv.QUOTE_MINIMAL)
        c.writerow(['ID', 'Prediction'])
        for i in range(0, len(ans)):
            c.writerow([i + 1, ans[i]])
    return 0

def listToMat(l):
    return np.array(l).reshape(-1, np.array(l).shape[-1])

def plotter(measured, predicted):
    # plot the result
    fig, ax = plt.subplots()
    y = measured
    ax.scatter(y, predicted)
    ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()
    return 0

def readFileDir(fileDir):
    dirFile = []
    for file in os.listdir(fileDir):
        if file.endswith(".nii"):
            dirFile.append(file)
    dirFile = listToMat(dirFile)
    numDir = dirFile.shape[1]
    if numDir == 0:
        return [],-1
    else:
        return dirFile, numDir

def readFileNumber(filename):
    startIndex = filename.find('_')
    endIndex = filename.find('.')
    if (startIndex != -1) & (endIndex > startIndex):
        return int(filename[startIndex + 1:endIndex])
    else:
        return -1
    
def computeStats(yy):
    temp = []
    temp.append(yy.mean())
    temp.append(yy.std())
#     sigma = yy.std()
    temp.append(np.median(yy))
    temp.append(yy.max())
    temp.append(yy.min())
    m3 = stats.moment(yy,3)
    temp.append(stats.skew(yy))
    m4 = stats.moment(yy,4)
    temp.append(stats.kurtosis(yy))
    temp.append(m3)
    temp.append(m4)
    
    return (temp)

def statsFeature(data):
    features = []
    #total = allBlock(data)
    #for i in range(len(total)):
    #    tempFeature = computeStats(np.reshape(total[i],[-1,]))
    #    for j in range(len(tempFeature)):
    #        features.append(tempFeature[j])
    #slice = np.zeros([data.shape[0], data.shape[1], data.shape[2]])
    for i in range(data.shape[0]):
        #slice[i] = data[:,:,i]
        tempFeature = computeStats(np.reshape(data[:,:,i],[-1,]))
         #print(len(tempFeature))
        for j in range(len(tempFeature)):
            features.append(tempFeature[j])     
        
    return features

def mriToFeature(fileDir):
    file, number = readFileDir(fileDir)
    numberStats = 9
    numberBlock = 176 # Z-axis
    features = np.zeros([number, numberStats*numberBlock])
    #print(number)
    # get the MRI image
    for i in range(number):
        filename = file[0,i]
        mriNumber = readFileNumber(filename)
        #print(mriNumber)
        img = nib.load(fileDir + filename)
        imgData = img.get_data()
        d2 = np.reshape(imgData, [imgData.shape[0],imgData.shape[1],imgData.shape[2]])
        #print(d2.shape, imgData.shape[0], imgData.shape[1],imgData.shape[2])
        features[mriNumber - 1,:] = statsFeature(d2)
    return features


In [22]:
from numpy import genfromtxt
# feature parameters 
numberStats = 9
numberBlock = 176 # Z-axis

# set the directories
testDir = '../testMRI/set_test/'
trainDir = '../testMRI/set_train/'
trainTarget = '../data/targets.csv'

# read the training targets
temp = []
temp.append(genfromtxt(trainTarget, delimiter=','))
trainingTargets = (listToMat(temp)).T

# testDir = '../data/set_test/'
testFile, numberTest = readFileDir(testDir)
testFeatures = np.zeros([numberTest, numberStats*numberBlock])

In [23]:
featureTest = mriToFeature(testDir)

In [19]:
featureTest.shape

(138, 1584)

In [20]:
csvFormatedOutput('../test.csv',testFeatures[0,:])

0